In [18]:
from collections.abc import Sequence
import json
import logging
from pathlib import Path
import re
from typing import Any

import h5py
import matplotlib.pyplot as plt
import nrrd
import numpy as np
import pydantic
from scipy.spatial.distance import euclidean
import vedo

from histalign.backend.io import gather_alignment_paths, load_image
from histalign.backend.maths import (
    apply_rotation,
    compute_centre,
    compute_normal,
    compute_normal_from_raw,
    compute_origin,
)
from histalign.backend.models import AlignmentSettings, VolumeSettings
from histalign.backend.registration import Registrator
from histalign.backend.registration.alignment import (
    replace_path_parts,
)

vedo.settings.default_backend = "vtk"

In [19]:
PathType = str | Path

ALIGNMENT_FILE_NAME_PATTERN = re.compile(r"[0-9a-f]{32}\.json")
_SUPPORTED_TYPES = [".h5", ".hdf5", ".nrrd", ".json"]


def imshow(image: np.ndarray) -> None:
    plt.imshow(image)
    plt.axis(False)
    plt.show()


def show(objects: object | Sequence[object], axes: int = 3) -> None:
    try:
        objects = [] + objects
    except TypeError:
        objects = [objects]

    vedo.show(
        objects,
        axes=3,
        interactive=False,
    ).interactive().close()


def load_file(path: PathType) -> Any:
    path = Path(path)
    if (suffix := path.suffix) not in _SUPPORTED_TYPES:
        raise ValueError(
            f"File extension not supported. Received: {suffix}. Allowed: {' '.join(_SUPPORTED_TYPES)}"
        )

    if suffix == ".json":
        data = json.load(path.open())
    elif suffix == ".nrrd":
        data = nrrd.read(path)[0]
    elif suffix in [".h5", ".hdf5"]:
        with h5py.File(path) as handle:
            data = handle[list(handle.keys())[0]][:]

    return data

In [20]:
atlas_path = "/home/ediun/.local/share/histalign/atlases/average_template_100.nrrd"
atlas_array = load_file(atlas_path)
atlas_volume = vedo.Volume(atlas_array)

### Building an alignment volume

1. Receive path to the alignment directory.
2. Collect alignment paths.
3. Loop over the paths.
    1. Load the alignment settings.
    2. Apply regex substitution.
    3. Load the file.
    4. Compute the origin and normal of the alignment.
    5. Check dimensionality.
    6. If 2D:
        1. Generate point cloud from origin, normal, and shape.
        2. Interpolate data into the master volume.
    7. If 3D:
        1. Compute origin for each Z-slice.
        2. Treat as-if 2D.

In [21]:
def build_point_cloud(
    origin: Sequence[float], shape: Sequence[int], settings: VolumeSettings
) -> vedo.Points:
    # Build a plane assuming no rotations
    plane = vedo.Plane(
        pos=(0, 0, 0),
        normal=compute_normal_from_raw(0, 0, settings.orientation),
        s=shape,
    )

    # Extract the four corners of the plane
    p0, p1, _, p3 = plane.points

    # Compute the normals of two orthogonal edges
    normal1 = (p0 - p1) / euclidean(p1, p0)
    normal2 = (p3 - p1) / euclidean(p1, p3)

    # Apply alignment rotation on normals
    normal1 = apply_rotation(normal1, settings)
    normal2 = apply_rotation(normal2, settings)

    # Generate a grid of coordinates the same size as the plane
    xs, ys = np.meshgrid(
        np.linspace(0, round(euclidean(p1, p0)), round(euclidean(p1, p0))),
        np.linspace(0, round(euclidean(p1, p3)), round(euclidean(p1, p3))),
    )
    points = np.vstack([xs.ravel(), ys.ravel()])

    # Apply alignment rotation on the points
    points = np.dot(np.vstack((normal1, normal2)).T, points).T

    # Translate the grid origin to the alignment origin
    points += -vedo.Points(points).center_of_mass() + origin

    return vedo.Points(points)

In [22]:
_module_logger = logging.getLogger(__name__)

alignment_directory = Path(
    "/home/ediun/histalign-projects/microns_100_coronal_3d_artificial/bb6ec0f5c8"
)
alignment_paths = gather_alignment_paths(alignment_directory)
channel_index = ""
channel_regex = ""
projection_regex = "_max"
misc_regexes = []
misc_subs = []

# Array inside which to store interpolated data from alignment point clouds
alignment_array = None
# Dummy volume used to query the grid coordinates when interpolating
query_volume = None
for alignment_path in alignment_paths:
    # Load the alignment settings
    settings = AlignmentSettings(**json.load(alignment_path.open()))

    # Apply regex substitution to the histology path
    substituted_path = replace_path_parts(
        settings.histology_path,
        channel_index,
        channel_regex,
        projection_regex,
        misc_regexes,
        misc_subs,
    )
    try:
        settings.histology_path = substituted_path
    except pydantic.ValidationError:
        _module_logger.warning(
            f"Histology path after regex substitution does not exist "
            f"(original is '{settings.histology_path}', "
            f"substituted is '{substituted_path}'). "
            f"Using the same projected image as was used during registration."
        )

    # Load the image array (allowed to be 2D or 3D)
    array = load_image(settings.histology_path, allow_stack=True)
    if len(array.shape) not in [2, 3]:
        _module_logger.error(
            "Only image arrays with 2 and 3 dimensions (XY and XYZ) are allowed."
        )
        continue

    # Compute the origin and normal as described by the alignment
    alignment_origin = compute_origin(
        compute_centre(settings.volume_settings.shape), settings.volume_settings
    )
    alignment_normal = compute_normal(settings.volume_settings)

    # List all the 2D images along their origins
    images = []
    origins = []
    # If 2D, only one image and origin
    if len(array.shape) == 2:
        images = [array]
        origins = [alignment_origin]
    # If 3D, extract each image and compute its origin
    else:
        slice_ = [slice(None)] * len(array.shape)
        # Assume the Z-dimension is the smallest one
        z_dimension_index = array.shape.index(min(array.shape))
        z_count = array.shape[z_dimension_index]

        # Loop over each Z-slice to extract the images
        for index in range(z_count):
            slice_[z_dimension_index] = index
            images.append(array[tuple(slice_)])

        # Loop over multiple of the normal to get origins
        for i in range(-int(z_count / 2) + (z_count % 2 == 0), z_count // 2 + 1):
            # TODO: Scale multiple by the real Z-spacing (currently assuming same as
            #       resolution).
            origins.append(alignment_origin + i * alignment_normal)

    # Register each image
    registrator = Registrator()
    for index, origin in enumerate(origins):
        image = registrator.get_forwarded_image(
            images[index], settings, origin.tolist()
        )
        images[index] = image

    # Loop over each image and generate its 3D point cloud
    point_clouds = []
    for image, origin in zip(images, origins):
        cloud = build_point_cloud(origin, image.shape, settings.volume_settings)

        # Insert point data from registered image
        cloud.pointdata["ImageScalars"] = image.flatten()

        point_clouds.append(cloud)

    # Interpolate the point clouds
    if alignment_array is None:
        alignment_array = np.zeros(settings.volume_settings.shape, dtype=np.uint16)
        query_volume = vedo.Volume(alignment_array)

    for points in point_clouds:
        # Interpolate and store the result in a temporary array
        tmp_array = query_volume.interpolate_data_from(points, radius=1).tonumpy()
        # tmp_array = query_volume.resample_data_from(points).tonumpy()
        tmp_array = np.round(tmp_array).astype(np.uint16)

        # TODO: Might be worth thinking of another way to merge. Using the maximum works
        #       fine when working with non-overlapping slices but a mean or something
        #       more robust might make more sense when tmp_array and
        #       interpolation_array have common, non-zero points.
        # Merge the new plane into the master array
        alignment_array[:] = np.maximum(alignment_array, tmp_array)

alignment_volume = vedo.Volume(alignment_array)

In [9]:
show(
    [
        alignment_volume,
        # *point_clouds,
        # atlas_volume.alpha(alpha=[0, 0.1]),
        # vedo.Points([alignment_origin], r=20, c="red"),
        # vedo.Points(origins, r=10, c="blue"),
    ]
)

# Backups

In [24]:
_module_logger = logging.getLogger(__name__)

alignment_directory = Path(
    "/home/ediun/histalign-projects/microns_100_coronal_3d_artificial/bb6ec0f5c8"
)
alignment_paths = gather_alignment_paths(alignment_directory)
# alignment_path = alignment_paths[0]
channel_index = ""
channel_regex = ""
projection_regex = "_max"
misc_regexes = []
misc_subs = []

# Array inside which to store interpolated data from alignment point clouds
alignment_array = None
# Dummy volume used to query the grid coordinates when interpolating
query_volume = None
for alignment_path in alignment_paths:
    ###################################
    ### Load the alignment settings ###
    ###################################
    settings = AlignmentSettings(**json.load(alignment_path.open()))

    ################################
    ### Apply regex substitution ###
    ################################
    substituted_path = replace_path_parts(
        settings.histology_path,
        channel_index,
        channel_regex,
        projection_regex,
        misc_regexes,
        misc_subs,
    )

    # Attempt replacing settings attribute
    try:
        settings.histology_path = substituted_path
    except pydantic.ValidationError:
        _module_logger.warning(
            f"Histology path after regex substitution does not exist "
            f"(original is '{settings.histology_path}', "
            f"substituted is '{substituted_path}'). "
            f"Using the same projected image as was used during registration."
        )

    #####################
    ### Load the file ###
    #####################
    array = load_image(settings.histology_path, allow_stack=True)

    if len(array.shape) not in [2, 3]:
        _module_logger.error(
            "Only image arrays with 2 and 3 dimensions (XY and XYZ) are allowed."
        )
        continue

    #####################################
    ### Compute the origin and normal ###
    #####################################
    alignment_origin = compute_origin(
        compute_centre(settings.volume_settings.shape), settings.volume_settings
    )
    alignment_normal = compute_normal(settings.volume_settings)

    ############################
    ### Check dimensionality ###
    ############################
    # List all the 2D images along their origins
    images = []
    origins = []
    # If 2D, only one image and origin
    if len(array.shape) == 2:
        images = [array]
        origins = [alignment_origin]
    # If 3D, separate each image and compute its origin
    else:
        slice_ = [slice(None)] * len(array.shape)
        # Assume the Z-dimension is the smallest one
        z_dimension_index = array.shape.index(min(array.shape))
        z_count = array.shape[z_dimension_index]

        # Loop over each Z-slice to extract the images
        for index in range(z_count):
            slice_[z_dimension_index] = index
            images.append(array[tuple(slice_)])

        # Loop over multiple of the normal to get origins
        for i in range(-int(z_count / 2) + (z_count % 2 == 0), z_count // 2 + 1):
            # TODO: Scale multiple by the real Z-spacing (currently assuming same as
            #       resolution.
            origins.append(alignment_origin + i * alignment_normal)

    ###########################
    ### Register each image ###
    ###########################
    registrator = Registrator()
    for index, origin in enumerate(origins):
        image = registrator.get_forwarded_image(
            images[index], settings, origin.tolist()
        )
        images[index] = image

    #############################
    ### Generate point clouds ###
    #############################
    # Loop over each image and generate its 3D point cloud
    point_clouds = []
    no_rotation_normal = compute_normal_from_raw(
        0, 0, settings.volume_settings.orientation
    )
    for image, origin in zip(images, origins):
        # Build a plane assuming no rotations
        plane = vedo.Plane(pos=(0, 0, 0), normal=no_rotation_normal, s=image.shape)

        # Extract the four corners of the plane
        p0, p1, _, p3 = plane.points

        # Compute the normals of two orthogonal edges
        normal1 = (p0 - p1) / euclidean(p1, p0)
        normal2 = (p3 - p1) / euclidean(p1, p3)

        # Apply alignment rotation on normals
        normal1 = apply_rotation(normal1, settings.volume_settings)
        normal2 = apply_rotation(normal2, settings.volume_settings)

        # Generate a grid of coordinates the same size as the plane
        xs, ys = np.meshgrid(
            np.linspace(0, round(euclidean(p1, p0)), round(euclidean(p1, p0))),
            np.linspace(0, round(euclidean(p1, p3)), round(euclidean(p1, p3))),
        )
        points = np.vstack([xs.ravel(), ys.ravel()])

        # Apply alignment rotation on the points
        points = np.dot(np.vstack((normal1, normal2)).T, points).T

        # Convert points to a vedo Points object to compute the centre of mass
        points_vedo = vedo.Points(points)

        # Translate the grid origin to the alignment origin
        points += -points_vedo.center_of_mass() + origin
        points_vedo = vedo.Points(points)

        # Insert point data from registered image
        points_vedo.pointdata["ImageScalars"] = image.flatten()

        point_clouds.append(points_vedo)

    # Interpolate the point clouds
    if alignment_array is None:
        alignment_array = np.zeros(settings.volume_settings.shape, dtype=np.uint16)
        query_volume = vedo.Volume(alignment_array)

    for points in point_clouds:
        # Interpolate and store the result in a temporary array
        tmp_array = query_volume.interpolate_data_from(points, radius=1).tonumpy()
        tmp_array = np.round(tmp_array).astype(np.uint16)

        # TODO: Might be worth thinking of another way to merge. Using the maximum works
        #       fine when working with non-overlapping slices but a mean or something
        #       more robust might make more sense when tmp_array and aligned_array have
        #       common, non-zero points.
        # Merge the new plane into the master array
        alignment_array[:] = np.maximum(alignment_array, tmp_array)

alignment_volume = vedo.Volume(alignment_array)

In [ ]:
# Validate arguments
if channel_regex is not None and channel_index is None:
    _module_logger.warning(
        "Received channel regex but no channel index. Building alignment "
        "volume using the same channel as was used for alignment."
    )
elif channel_regex is None and channel_index is not None:
    _module_logger.warning(
        "Received channel index but no channel regex. Building alignment "
        "volume using the same channel as was used for alignment."
    )

_module_logger.debug(
    f"Building alignment volume for directory '{alignment_directory}'."
)

# Gather all the alignment settings paths
alignment_paths = gather_alignment_paths(alignment_directory)
if not alignment_paths:
    _module_logger.error(f"No alignments found for directory '{alignment_directory}'.")
    return

_module_logger.debug(f"Found {len(alignment_paths)} alignments.")

# Inspect cache
# TODO: Improve cache path so that it takes into account regexes
cache_directory = alignment_directory / "volumes" / "aligned"
os.makedirs(cache_directory, exist_ok=True)
cache_path = cache_directory / f"{alignment_directory.name}.h5"
if cache_path.exists() and not force:
    return
# Array inside which to store interpolated data from alignment point clouds
alignment_array = None
# Dummy volume used to query the grid coordinates when interpolating
query_volume = None
for progress_index, alignment_path in enumerate(alignment_paths):
    if (progress := progress_index + 1) % 5 == 0:
        _module_logger.debug(
            f"Gathered {progress}/{len(alignment_paths)} slices ({progress / len(alignment_paths):.0%})."
        )

    # Load the alignment settings
    settings = AlignmentSettings(**json.load(alignment_path.open()))

    # Apply regex substitution to the histology path
    substituted_path = replace_path_parts(
        settings.histology_path,
        channel_index,
        channel_regex,
        projection_regex,
        misc_regexes,
        misc_subs,
    )
    try:
        settings.histology_path = substituted_path
    except pydantic.ValidationError:
        _module_logger.warning(
            f"Histology path after regex substitution does not exist for "
            f"'{settings.histology_path}' (substituted: '{substituted_path}'). "
            f"Using the same projected image as was used during registration."
        )

    # Load the image array (allowed to be 2D or 3D)
    array = load_image(settings.histology_path, allow_stack=True)
    if len(array.shape) not in [2, 3]:
        _module_logger.error(
            "Only image arrays with 2 and 3 dimensions (XY and XYZ) are allowed."
        )
        continue

    # Compute the origin and normal as described by the alignment
    alignment_origin = compute_origin(
        compute_centre(settings.volume_settings.shape), settings.volume_settings
    )
    alignment_normal = compute_normal(settings.volume_settings)

    # List all the 2D images along their origins
    images = []
    origins = []
    # If 2D, only one image and origin
    if len(array.shape) == 2:
        images = [array]
        origins = [alignment_origin]
    # If 3D, extract each image and compute its origin
    else:
        slice_ = [slice(None)] * len(array.shape)
        # Assume the Z-dimension is the smallest one
        z_dimension_index = array.shape.index(min(array.shape))
        z_count = array.shape[z_dimension_index]

        # Loop over each Z-slice to extract the images
        for index in range(z_count):
            slice_[z_dimension_index] = index
            images.append(array[tuple(slice_)])

        # Loop over multiple of the normal to get origins
        for i in range(-int(z_count / 2) + (z_count % 2 == 0), z_count // 2 + 1):
            # TODO: Scale multiple by the real Z-spacing (currently assuming same as
            #       resolution).
            origins.append(alignment_origin + i * alignment_normal)

    # Register each image
    registrator = Registrator()
    for index, origin in enumerate(origins):
        image = registrator.get_forwarded_image(
            images[index], settings, origin.tolist()
        )
        images[index] = image

    # Loop over each image and generate its 3D point cloud
    point_clouds = []
    for image, origin in zip(images, origins):
        cloud = build_point_cloud(origin, image.shape, settings.volume_settings)

        # Insert point data from registered image
        cloud.pointdata["ImageScalars"] = image.flatten()

        point_clouds.append(cloud)

    # Interpolate the point clouds
    if alignment_array is None:
        alignment_array = np.zeros(settings.volume_settings.shape, dtype=np.uint16)
        query_volume = vedo.Volume(alignment_array)

    for points in point_clouds:
        # Interpolate and store the result in a temporary array
        tmp_array = query_volume.interpolate_data_from(points, radius=1).tonumpy()
        # tmp_array = query_volume.resample_data_from(points).tonumpy()
        tmp_array = np.round(tmp_array).astype(np.uint16)

        # TODO: Might be worth thinking of another way to merge. Using the maximum works
        #       fine when working with non-overlapping slices but a mean or something
        #       more robust might make more sense when tmp_array and
        #       interpolation_array have common, non-zero points.
        # Merge the new plane into the master array
        alignment_array[:] = np.maximum(alignment_array, tmp_array)

_module_logger.debug(f"Finished gathering slices. Caching result to '{cache_path}'.")
with h5py.File(cache_path, "w") as handle:
    handle.create_dataset(name="array", data=alignment_array, compression="gzip")
append_volume(alignment_directory, cache_path, "aligned")